In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as skl
import pandas as pd
%matplotlib inline

C:\Users\H\Anaconda3\envs\MSBD5001\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [19]:
# loading training/testing data
all_data = pd.read_csv("preprocessed_data.csv")
all_data = shuffle(all_data)
X = all_data.drop(["Survived"], axis=1)
y = all_data["Survived"]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)
print(len(X_train))
print(len(X_test))

712
179


In [27]:
model = skl.LogisticRegression()
model.fit(X_train, y_train)
predict_y = model.predict(X_train)
print("training accuracy:", (predict_y == y_train).sum()/ len(predict_y))


predict_y = model.predict(X_test)
print("testing accuracy:", (predict_y == y_test).sum()/ len(predict_y))

training accuracy: 0.8160112359550562
testing accuracy: 0.7541899441340782


C:\Users\H\Anaconda3\envs\MSBD5001\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [28]:
model.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'warn',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'warn',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [42]:
# Set the parameters by cross-validation
tuned_parameters = [{'solver': ['lbfgs'], 'penalty': ['l2'], 'C': [0.1,1, 10, 100]},
                   {'solver':['liblinear'], 'penalty': ['l2','l1'], 'C': [0.1,1, 10, 100]}]


clf = GridSearchCV(skl.LogisticRegression(), tuned_parameters, cv=5,
                   scoring='precision_macro')
clf.fit(X_train, y_train)

print(clf.best_params_)
print(clf.best_score_ )
print(clf.cv_results_['mean_test_score'])
print(clf.cv_results_['std_test_score'])
print(clf.cv_results_["params"])

{'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.7998757842006038
[0.79802013 0.79723287 0.79293453 0.79293453 0.79987578 0.78433466
 0.79723287 0.79362852 0.79293453 0.79293453 0.79293453 0.79293453]
[0.0443576  0.04434993 0.03813677 0.03813677 0.04558246 0.02895295
 0.04434993 0.03875878 0.03813677 0.03813677 0.03813677 0.03813677]
[{'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}, {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}, {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}, {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}, {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}, {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}, {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}, {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}, {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}, {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}, {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}, {'C': 100, 'penalty': 'l1', 'solver': 'liblinear'}]


In [43]:
print("Detailed classification report:")

y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       0.76      0.80      0.78        97
           1       0.75      0.71      0.73        82

    accuracy                           0.76       179
   macro avg       0.76      0.76      0.76       179
weighted avg       0.76      0.76      0.76       179


